In [52]:
import umap
import pandas as pd
import os

raw_dir = './raws'
ex_dir = './processed'

train = pd.read_csv(os.path.join(raw_dir, 'train.csv')).reset_index()
greeks = pd.read_csv(os.path.join(raw_dir, 'greeks.csv'))

train.replace(['A', 'B'], [0, 1], inplace=True)
train.fillna(train.loc[:, 'AB':'GL'].median(), inplace=True)
train

,index,Id,AB,AF,AH,AM,AR,AX,AY,AZ,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [54]:
# 次元削減
import umap

rand = 0
nc = 19
nn = 10

um = umap.UMAP(n_components=nc, n_neighbors=nn, random_state=rand)
x = train.drop(columns=['Id', 'Class'])

um.fit(x)
x_rd = pd.DataFrame(um.transform(x)).reset_index()

# greeks.csvからのクラス抽出
x_rd = pd.concat([train.loc[:, ['Id', 'Class']], x_rd], axis=1)
x_rd.drop(columns='index', inplace=True)

x_rd = x_rd.merge(greeks.loc[:, 'Id':'Alpha'], on='Id')
l = x_rd['Alpha'].tolist()

x_rd['B'] = [1 if i=='B' else 0 for i in l]
x_rd['C'] = [1 if i=='C' else 0 for i in l]
x_rd['D'] = [1 if i=='D' else 0 for i in l]
x_rd.drop(columns='Alpha', inplace=True)

# 保存
x_rd.to_csv(os.path.join(ex_dir, 'train.csv'))
x_rd

,Id,Class,0,1,2,3,4,5,6,7,...,12,13,14,15,16,17,18,B,C,D
0,000ff2bfdfe9,1,11.445144,5.433045,6.721895,6.682347,5.640959,5.840046,4.565876,4.111125,...,4.400391,3.868697,5.381531,4.418231,5.283762,5.035281,6.355948,1,0,0
1,007255e47698,0,4.325884,9.005783,9.432169,4.162204,3.437424,5.055123,4.785954,4.585079,...,4.020988,3.861433,5.483879,4.709369,6.124230,4.936836,6.445664,0,0,0
2,013f2bd269f5,0,9.076456,7.124766,9.228440,4.990589,3.906366,5.010014,5.154222,4.626291,...,3.563549,4.021563,5.057630,4.686233,5.323278,4.445773,6.230271,0,0,0
3,043ac50845d5,0,10.756622,5.725919,8.387392,6.167576,4.970712,5.507069,4.892031,4.531415,...,4.320321,4.347462,5.703804,4.457173,4.764696,4.856787,5.988841,0,0,0
4,044fb8a146ec,1,8.961798,6.394759,6.766585,5.759810,5.139689,5.942494,4.750665,4.230611,...,4.308183,3.660025,5.597006,4.567157,5.620451,5.211353,6.465797,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0,8.849371,6.462133,6.801899,5.690662,5.098525,5.933166,4.760657,4.247728,...,4.288914,3.656954,5.605077,4.569067,5.646219,5.211470,6.478902,0,0,0
613,fd895603f071,0,9.040792,6.223179,6.262098,5.937671,5.384762,6.024131,4.707580,4.169101,...,4.376979,3.597477,5.693288,4.525637,5.663247,5.238584,6.522917,0,0,0
614,fd8ef6377f76,0,11.840574,5.945441,8.795070,6.515954,4.788178,4.978555,4.753714,4.432634,...,3.947263,4.293802,5.197683,4.392166,4.943607,4.553967,6.117896,0,0,0
615,fe1942975e40,0,12.769028,5.137832,7.064750,7.106809,5.729769,5.686036,4.553656,3.957609,...,4.274430,3.986098,5.076440,4.256080,5.179341,4.801494,6.404760,0,0,0
